In [1]:
from src import *

## Description



GOAL: 

## Table of Contents


In [13]:
mri_path = "./data/fmri_compile.csv"
mri_df   = pd.read_csv(mri_path)

Single Video Acquisition

In [16]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')

filename_movie = 'TC_400_sub-S09_ses-4_Sintel.csv'
filename_rest  = 'TC_400_sub-S23_ses-1_Rest.csv'

movie_series, movie_df = df_to_timeseries(mri_df, filename_movie)
rest_series, rest_df   = df_to_timeseries(mri_df, filename_rest)

fc_movie = correlation_measure.fit_transform([movie_series])[0]
fc_rest  = correlation_measure.fit_transform([rest_series])[0]